Augmentation

-- Preprocess action tokens in raw, unlabeled trajectories so that LMs (explainer, baseline...) can make predictions to label them with instructions.

In [1]:
import os
os.environ['ALFRED_ROOT'] = '/root/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

In [2]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.data = '/root/data_alfred/unlabeled_12k_20201206/seen/'
args.splits = '/root/data_alfred/splits/debug_20201210.json'
args.pframe = 300
args.fast_epoch = False

## SPLITS

In [3]:
with open(args.splits, 'r') as f:
    splits = json.load(f)

In [4]:
len(splits['augmentation'])

198

In [5]:
splits['augmentation'][:2]

[{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373',
  'repeat_idx': 0,
  'full_traj_success': True,
  'collected_subgoals': 6},
 {'task': 'pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425',
  'repeat_idx': 0,
  'full_traj_success': True,
  'collected_subgoals': 7}]

## EXPLAINER

In [6]:
model_name = 'model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2'
EXPLAINER = f'/root/home/hoyeung/blob_alfred_data/exp_all/{model_name}/net_epoch_32.pth'
args.pp_folder = 'pp_' + model_name

In [7]:
ckpt = torch.load(EXPLAINER)
ckpt['vocab']

{'action_high': Vocab(93), 'word': Vocab(2360), 'action_low': Vocab(15)}

In [8]:
dataset = Dataset(args, ckpt['vocab'])
dataset.preprocess_splits(splits, preprocess_lang=False, train_vocab=False, save_vocab_to_dout=False, augmentation_mode=True)

  3% (6 of 198) |                        | Elapsed Time: 0:00:00 ETA:   0:00:06

Preprocessing augmentation
[{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_clean_then_place_in_recep-Spoon-None-CounterTop-11/trial_T20190913_132015_402664', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_and_place_with_movable_recep-CellPhone-Plate-Shelf-211/trial_T20190909_182029_502585', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 8}, {'task': 'pick_clean_then_place_in_recep-Lettuce-None-DiningTable-23/trial_T20190912_150010_333678', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_two_obj_and_place-RemoteControl-None-SideTable-220/trial_T20190911_203801_914153', 'repeat_idx': 0, 'full_traj_succ

100% (198 of 198) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


#### Explainer Output Sanity Check

In [9]:
# results for each trajectory is saved to args.data/<task name>/<trial name>/pp_<model name>
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/

total 28344
-rw-r--r-- 1 root root 24486244 Dec  9 14:27 feat_conv.pt
-rw-r--r-- 1 root root  4475639 Dec 27 18:15 metadata_states.json
drwxr-xr-x 2 root root       28 Sep 19  2019 pddl_states
drwxr-xr-x 2 root root       24 Dec 27 18:35 pp_model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs
drwxr-xr-x 2 root root       24 Dec 27 18:32 pp_model_seq2seq_per_subgoal,name_v2_epoch_35_baseline
drwxr-xr-x 2 root root       24 Dec 27 18:23 pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2
drwxr-xr-x 2 root root       24 Dec 27 18:27 pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_axu_loss_none
drwxr-xr-x 2 root root     8192 Nov 30 10:44 raw_images
-rw-r--r-- 1 root root    41547 Sep 19  2019 traj_data.json


In [10]:
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2

total 368
-rw-r--r-- 1 root root 375953 Dec 27 18:41 ann_0.json


In [11]:
with open('/root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2/ann_0.json', 'r') as f:
    ann = json.load(f)

In [12]:
ann['num']['action_low']

[[{'action': 3,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 5,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 5,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
 

## EXPLAINER AUX LOSS ONLY

In [13]:
model_name = 'model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_axu_loss_none'
EXPLAINER_AUX_ONLY = f'/root/home/hoyeung/blob_alfred_data/exp_all/{model_name}/net_epoch_27.pth'
args.pp_folder = 'pp_' + model_name

In [14]:
ckpt = torch.load(EXPLAINER_AUX_ONLY)
ckpt['vocab']

{'action_high': Vocab(93), 'word': Vocab(2360), 'action_low': Vocab(15)}

In [15]:
dataset = Dataset(args, ckpt['vocab'])
dataset.preprocess_splits(splits, preprocess_lang=False, train_vocab=False, save_vocab_to_dout=False, augmentation_mode=True)

  3% (6 of 198) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04

Preprocessing augmentation
[{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_clean_then_place_in_recep-Spoon-None-CounterTop-11/trial_T20190913_132015_402664', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_and_place_with_movable_recep-CellPhone-Plate-Shelf-211/trial_T20190909_182029_502585', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 8}, {'task': 'pick_clean_then_place_in_recep-Lettuce-None-DiningTable-23/trial_T20190912_150010_333678', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_two_obj_and_place-RemoteControl-None-SideTable-220/trial_T20190911_203801_914153', 'repeat_idx': 0, 'full_traj_succ

100% (198 of 198) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


In [16]:
# results for each trajectory is saved to args.data/<task name>/<trial name>/pp_<model name>
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/

total 28344
-rw-r--r-- 1 root root 24486244 Dec  9 14:27 feat_conv.pt
-rw-r--r-- 1 root root  4475639 Dec 27 18:15 metadata_states.json
drwxr-xr-x 2 root root       28 Sep 19  2019 pddl_states
drwxr-xr-x 2 root root       24 Dec 27 18:35 pp_model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs
drwxr-xr-x 2 root root       24 Dec 27 18:32 pp_model_seq2seq_per_subgoal,name_v2_epoch_35_baseline
drwxr-xr-x 2 root root       24 Dec 27 18:23 pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2
drwxr-xr-x 2 root root       24 Dec 27 18:27 pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_axu_loss_none
drwxr-xr-x 2 root root     8192 Nov 30 10:44 raw_images
-rw-r--r-- 1 root root    41547 Sep 19  2019 traj_data.json


## BASELINE

In [17]:
model_name = 'model_seq2seq_per_subgoal,name_v2_epoch_35_baseline'
BASELINE = f'/root/home/hoyeung/blob_alfred_data/exp_all/{model_name}/net_epoch_29.pth'
args.pp_folder = 'pp_' + model_name

In [18]:
ckpt = torch.load(BASELINE)
ckpt['vocab']

{'action_high': Vocab(93), 'word': Vocab(2360), 'action_low': Vocab(15)}

In [19]:
dataset = Dataset(args, ckpt['vocab'])
dataset.preprocess_splits(splits, preprocess_lang=False, train_vocab=False, save_vocab_to_dout=False, augmentation_mode=True)

  3% (6 of 198) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04

Preprocessing augmentation
[{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_clean_then_place_in_recep-Spoon-None-CounterTop-11/trial_T20190913_132015_402664', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_and_place_with_movable_recep-CellPhone-Plate-Shelf-211/trial_T20190909_182029_502585', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 8}, {'task': 'pick_clean_then_place_in_recep-Lettuce-None-DiningTable-23/trial_T20190912_150010_333678', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_two_obj_and_place-RemoteControl-None-SideTable-220/trial_T20190911_203801_914153', 'repeat_idx': 0, 'full_traj_succ

100% (198 of 198) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


In [20]:
# results for each trajectory is saved to args.data/<task name>/<trial name>/pp_<model name>
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/

total 28344
-rw-r--r-- 1 root root 24486244 Dec  9 14:27 feat_conv.pt
-rw-r--r-- 1 root root  4475639 Dec 27 18:15 metadata_states.json
drwxr-xr-x 2 root root       28 Sep 19  2019 pddl_states
drwxr-xr-x 2 root root       24 Dec 27 18:35 pp_model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs
drwxr-xr-x 2 root root       24 Dec 27 18:32 pp_model_seq2seq_per_subgoal,name_v2_epoch_35_baseline
drwxr-xr-x 2 root root       24 Dec 27 18:23 pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2
drwxr-xr-x 2 root root       24 Dec 27 18:27 pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_axu_loss_none
drwxr-xr-x 2 root root     8192 Nov 30 10:44 raw_images
-rw-r--r-- 1 root root    41547 Sep 19  2019 traj_data.json


## GOAL EXPLAINER

In [21]:
model_name = 'model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs'
GOAL_EXPLAINER = f'/root/home/hoyeung/blob_alfred_data/exp_all/{model_name}/net_epoch_10.pth'
args.pp_folder = 'pp_' + model_name

In [22]:
ckpt = torch.load(GOAL_EXPLAINER)
ckpt['vocab']

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}

In [23]:
dataset = Dataset(args, ckpt['vocab'])
dataset.preprocess_splits(splits, preprocess_lang=False, train_vocab=False, save_vocab_to_dout=False, augmentation_mode=True)

  3% (6 of 198) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04

Preprocessing augmentation
[{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_clean_then_place_in_recep-Spoon-None-CounterTop-11/trial_T20190913_132015_402664', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 7}, {'task': 'pick_and_place_with_movable_recep-CellPhone-Plate-Shelf-211/trial_T20190909_182029_502585', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 8}, {'task': 'pick_clean_then_place_in_recep-Lettuce-None-DiningTable-23/trial_T20190912_150010_333678', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}, {'task': 'pick_two_obj_and_place-RemoteControl-None-SideTable-220/trial_T20190911_203801_914153', 'repeat_idx': 0, 'full_traj_succ

100% (198 of 198) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


In [24]:
# results for each trajectory is saved to args.data/<task name>/<trial name>/pp_<model name>
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/

total 28344
-rw-r--r-- 1 root root 24486244 Dec  9 14:27 feat_conv.pt
-rw-r--r-- 1 root root  4475639 Dec 27 18:15 metadata_states.json
drwxr-xr-x 2 root root       28 Sep 19  2019 pddl_states
drwxr-xr-x 2 root root       24 Dec 27 18:35 pp_model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs
drwxr-xr-x 2 root root       24 Dec 27 18:32 pp_model_seq2seq_per_subgoal,name_v2_epoch_35_baseline
drwxr-xr-x 2 root root       24 Dec 27 18:23 pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2
drwxr-xr-x 2 root root       24 Dec 27 18:27 pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_axu_loss_none
drwxr-xr-x 2 root root     8192 Nov 30 10:44 raw_images
-rw-r--r-- 1 root root    41547 Sep 19  2019 traj_data.json


In [25]:
!ls -l /root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-Knife-Pot-CounterTop-1/trial_T20190918_220230_284512/pp_model_seq2seq_nl_with_frames,name_v1.5_epoch_50_high_level_instrs

total 368
-rw-r--r-- 1 root root 375953 Dec 27 18:41 ann_0.json
